# Description of sample

We will create an AI application using next generation search capabilities of Azure AI Search (formerly Azure Cognitive Search). While there are several examples of using AI Search with PaaS database data sources, we will use a SQL Server as our data source. The SQL Servers lives within a private AVS environment. The SQL Server is running on a virtual machine and the connection is exposed through Azure Application Gateway. The SQL server has an SSL connection certificate installed and the connection calls the SQL Server through the domain name listed on that certificate.

The end behavior will be something like:

```
[User search]: Ballons
[AI Response]: After searching through our product database, here is what customers are saying: <text>
```

Behind the scenes, we take the following steps:
* Install an SSL certificate on the SQL Server
* Set up a sample table in a SQL Server and upload data to it
* Set up an index in Azure Cognitive Search to store the data we need, including vectorized versions of the text reviews
* Set up an indexer in Azure Cognitive Search to pull data into the index 
  * Automatically chunks and vectorizes the data using an Azure OpenAI Embedding service
* Use Azure Cognitive Search to process the user's query and search for the most relevant data
* Use an Azure OpenAI Completion service to respond to the user's query

Copyright (c) Microsoft Corporation.
Licensed under the MIT license.

## Requirements

This sample uses preview features from the [azure-search-documents](https://pypi.org/project/azure-search-documents/#description) package that has not been published on pypi. If you would like to use these preview features, please open a support request on the Search Service resource in the Azure portal, and we will provide instructions.

You will also need:
* An SSL certificate installed on the SQL Server
* SQL Server with server name, DB name, username, and password copied into `example.env`
  * The user must have permission to create a new table and enable and view change tracking on the database
* An OpenAI resource with the endpoint and key copied into `example.env`
* A Azure AI Search resource with the endpoint and key copied into `example.env`
* The Python packages listed in `requirements.txt` (can be installed using `pip`)
* The Microsoft ODBC 18 driver, [instructions here](https://learn.microsoft.com/en-us/sql/connect/odbc/microsoft-odbc-driver-for-sql-server?view=sql-server-ver16).


# Load environment variables and keys

In [277]:
from dotenv import dotenv_values
# specify the name of the .env file name 
env_name = "example.env"
config = dotenv_values(env_name)

In [278]:
# Load SQL Server database connection details
server = config["server"]
database = config["database"] 
username = config["username"] 
password = config["password"] 
driver = '{ODBC Driver 18 for SQL Server}'

In [279]:
# Load Open AI deployment details
import openai
openai.api_type = config["openai_api_type"]
openai.api_key = config['openai_api_key']
openai.api_base = config['openai_api_base']
openai.api_version = config['openai_api_version'] 
openai_deployment = config["openai_deployment_embedding"]
EMBEDDING_LENGTH = 1536

In [280]:
# Load Cognitive Search service details
cogsearch_key = config["cogsearch_api_key"]
service_endpoint = config["cogsearch_endpoint"]
index_name = config["cogsearch_index_name"] # Desired name of index -- does not need to exist already


# Upload data to SQL Server

### Connect to database

For simplicity, we set `autocommit=True` in the pyodbc connection parameters, which allows us to execute `ALTER` statements. In a production scenario, setting `autocommit=True` is not recommended; instead, the `ALTER` statements can be executed in SSMS or similar.

If a timeout error occurs, retry the cell.

## Create table

In [281]:
import pyodbc

# Create a connection string
conn_str = f"DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}"

# Establish a connection to the SQL Server
conn = pyodbc.connect(conn_str, autocommit=True)
cursor = conn.cursor()

### Create a database

Create the new database, if not exists

In [282]:
cursor.execute(f"IF NOT EXISTS (SELECT * FROM sys.databases WHERE name = '{database}') BEGIN CREATE DATABASE {database} END")
conn.commit()


### Create a table in the database

We will create a new table "foodreview" and upload the data from a csv file. We include a primary key, which is necessary for change tracking.

In [283]:
table_name = "partysupplies" 

# Drop previous table of same name if one exists
cursor.execute(f"DROP TABLE IF EXISTS {table_name};")
print("Finished dropping table (if existed)")

# Create a table
cursor.execute(f"""
               CREATE TABLE {table_name} 
               (Id int NOT NULL, 
               CONSTRAINT PK_{table_name}_Id PRIMARY KEY CLUSTERED (Id), 
               ProductId text, 
               UserId text, 
               ProfileName text, 
               Score integer,
               Time bigint,
               Summary text, 
               Text text,
               TextConcat text);
               """)
print("Finished creating table")

# Create a index
cursor.execute(f"CREATE INDEX idx_Id ON {table_name}(Id);")
print("Finished creating index")

Finished dropping table (if existed)
Finished creating table
Finished creating index


### Enable change tracking

This allows us to automatically update the index when changes are made to the data.

In [284]:
try:
    cursor.execute(f"ALTER DATABASE {database} SET CHANGE_TRACKING = ON (CHANGE_RETENTION = 2 DAYS, AUTO_CLEANUP = ON)")
except Exception as e:
    print(e)

('42000', "[42000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Change tracking is already enabled for database 'party_supplies'. (5088) (SQLExecDirectW)")


In [285]:
try:
    cursor.execute(f"ALTER TABLE {table_name} ENABLE CHANGE_TRACKING WITH (TRACK_COLUMNS_UPDATED = ON)")
except Exception as e:
    print(e)

## Upload data

### Load data from CSV

The data contains a few product reviews, with related info.

In [286]:
## Load Data
import numpy as np
import pandas as pd
df_all = pd.read_csv('../DataSet/partysupplies.csv')

df_all.head(3)

ParserError: Error tokenizing data. C error: Expected 8 fields in line 16, saw 9


### Manipulate data

For our example, we will combine the user's summary with the user's review text.

In [ ]:
df_all["TextConcat"] = df_all.apply(lambda row: f"Summary: {row['Summary']} | Review: {row['Text']}",
                                    axis = 1)

df_all.head(3)

,Id,ProductId,UserId,ProfileName,Score,Time,Summary,Text,TextConcat
0,1,P001,ABG53EBZBKD23,Julie Rogers,5,1333929600,Very happy with the quality,These balloons were perfect for our party. The...,Summary: Very happy with the quality | Review:...
1,2,P002,A1FD9E5C06UB6B,Margaret Murphy,3,1301011200,Highly recommend for parties,The streamers added a great touch to our decor...,Summary: Highly recommend for parties | Review...
2,3,P003,AOVROBZ8BNTP7,Keith Powers,4,1163376000,Kids had a blast,Napkins were sturdy and absorbed spills well. ...,Summary: Kids had a blast | Review: Napkins we...


### Upload to DB

In [ ]:
# Split the dataframe into batches
batch_size = 30
batches = [df_all[i:i + batch_size] for i in range(0, len(df_all), batch_size)]

#Iterate over each batch and insert the data into the database
for batch in batches:
    # Convert the batch dataframe to a list of tuples for bulk insertion
    rows = [tuple(row) for row in batch.itertuples(index=False)]
    
    # Define the SQL query for bulk insertion
    query = f"INSERT INTO {table_name} (Id, ProductId, UserId, ProfileName, Score, Time, Summary, Text, TextConcat) \
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)"
    cursor.executemany(query, rows)

## Example query

This checks that the data was uploaded correctly. We should have 99 rows at this point.

In [ ]:
# Execute the SELECT statement
try:
    cursor.execute(f"SELECT count(Id) FROM {table_name};")
    rows = cursor.fetchall()
    for row in rows:
        print(row)
except Exception as e:
    print(f"Error executing SELECT statement: {e}")

(300,)


## Commit changes

In [ ]:
cursor.commit()
cursor.close()

# Set up data source connection in Azure AI Search

## Import needed AI Search functions

In [ ]:
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient
from azure.search.documents.models import VectorizedQuery
from azure.search.documents.indexes.models import SemanticSearch
from azure.search.documents.models import (
    QueryAnswerType,
    QueryCaptionType,
    QueryLanguage,
    QueryType,
    VectorizedQuery,
    VectorQuery,
    VectorizableTextQuery,
    VectorFilterMode,    
)
from azure.search.documents.indexes.models import (  
    AzureOpenAIEmbeddingSkill,  
    AzureOpenAIParameters,  
    AzureOpenAIVectorizer,  
    ExhaustiveKnnParameters,  
    ExhaustiveKnnAlgorithmConfiguration,
    FieldMapping,  
    HnswParameters,  
    HnswAlgorithmConfiguration,
    IndexProjectionMode,  
    InputFieldMappingEntry,
    MergeSkill,
    OutputFieldMappingEntry,  
    SemanticPrioritizedFields,    
    SearchField,  
    SearchFieldDataType,  
    SearchIndex,  
    SearchIndexer,  
    SearchIndexerDataContainer,  
    SearchIndexerDataSourceConnection,  
    SearchIndexerIndexProjectionSelector,  
    SearchIndexerIndexProjections,  
    SearchIndexerIndexProjectionsParameters,  
    SearchIndexerSkillset,  
    SemanticConfiguration,  
    SemanticField,  
    SemanticSearch,  
    SplitSkill,  
    SqlIntegratedChangeTrackingPolicy,
    VectorSearch,
    VectorSearchProfile,
    VectorSearchAlgorithmKind,  
    VectorSearchAlgorithmMetric,  
    VectorSearchProfile,  
)  

## Create data source connection

This step creates a connection that will be used to pull data from our SQL table.

Documentation can be found [here.](https://learn.microsoft.com/en-us/azure/search/search-howto-connecting-azure-sql-database-to-azure-search-using-indexers)

In [ ]:
ds_conn_str = f'Encrypt=True;TrustServerCertificate=True;Connection Timeout=30;Server=tcp:{server};Database={database};User ID={username};Password={password};'

cogsearch_credential = AzureKeyCredential(cogsearch_key)
ds_client = SearchIndexerClient(service_endpoint, cogsearch_credential)
container = SearchIndexerDataContainer(name=table_name)

change_detection_policy = SqlIntegratedChangeTrackingPolicy()

data_source_connection = SearchIndexerDataSourceConnection(
    name=f"{index_name}-sqlserver-connection",
    type="azuresql",
    connection_string=ds_conn_str,
    container=container,
    data_change_detection_policy=change_detection_policy
)
data_source = ds_client.create_or_update_data_source_connection(data_source_connection)

print(f"Data source '{data_source.name}' created or updated")

Data source 'sqlserver-index-test-sqlserver-connection' created or updated


# Set up automatic chunking + vectorization + indexing

## Create index

The plan is:
1. Take the combined text (summary + review text) from each product review
2. Split the combined text into chunks
3. Embed each chunk as a vector
4. (Later) search for the most relevant chunk based on the incoming query. 

To enable this, the search index will store all of the following data, for each chunk of text:
* Id of chunk
* Chunk text
* Vector version of chunk text
* Id of parent row
* Product Id from parent row
* Review text from parent row
* Summary text from parent row
* Score from parent row

All of these values will be stored in SearchFields specified in the code below.

In this step we also configure the search algorithm(s), and the vectorizer that will automatically vectorize the incoming query.

Documentation about creating indexes can be found [here.](https://learn.microsoft.com/en-us/azure/search/search-how-to-create-search-index?tabs=index-other-sdks)

In [ ]:
# Create a search index
index_client = SearchIndexClient(
    endpoint=service_endpoint, credential=cogsearch_credential)

fields = [
    # Properties of individual chunk
    SearchField(name="Id", type=SearchFieldDataType.String, key=True,
                sortable=True, filterable=True, facetable=True, analyzer_name="keyword"),
    SearchField(name="chunk", type=SearchFieldDataType.String, sortable=False, filterable=False, facetable=False),
    SearchField(name="vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single), 
                vector_search_dimensions=EMBEDDING_LENGTH, vector_search_profile_name="my-vector-search-profile"),
    # Properties of original row in DB that the chunk belonged to
    SearchField(name="parent_id", type=SearchFieldDataType.String, sortable=True, filterable=True, facetable=True),
    SearchField(name="parent_product_id", type=SearchFieldDataType.String, sortable=True, filterable=True, facetable=True),
    SearchField(name="parent_text", type=SearchFieldDataType.String, sortable=True, filterable=True, facetable=True),
    SearchField(name="parent_summary", type=SearchFieldDataType.String, sortable=True, filterable=True, facetable=True),
    SearchField(name="parent_score", type=SearchFieldDataType.Int64, sortable=True, filterable=True, facetable=True)
]

# Configure the vector search configuration  
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(
            name="my-hnsw-config",
            kind=VectorSearchAlgorithmKind.HNSW
        )
    ],
    profiles=[
        VectorSearchProfile(
            name="my-vector-search-profile",
            algorithm_configuration_name="my-hnsw-config",
            vectorizer="my-openai"
        )
    ],
    vectorizers=[
        AzureOpenAIVectorizer(
            name="my-openai",
            kind="azureOpenAI",
            azure_open_ai_parameters=AzureOpenAIParameters(
                resource_uri=openai.api_base,
                deployment_id=openai_deployment,
                api_key=openai.api_key
            )
        )  
    ]  
)

semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        content_fields=[SemanticField(field_name="Id")]
    )
)

# Create the semantic settings with the configuration
semantic_settings = SemanticSearch(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields,
                    vector_search=vector_search, semantic_settings=semantic_settings)
result = index_client.create_or_update_index(index)
print(f'{result.name} created')

semantic_settings is not a known attribute of class <class 'azure.search.documents.indexes.models._index.SearchIndex'> and will be ignored


sqlserver-index-test created


## Create skillset

We use two pre-built skills:
1. The Split Skill takes the concatenated text and divides it into chunks (to stay within the token limits for the OpenAI embedding service).
2. The Azure Open AI Embedding Skill takes the outputs of the Split Skill and vectorizes them individually.

Afterwards, we apply an Index Projector to make it so that our final index has one item for every chunk of text (rather than one item for every original row in the DB).

We recommend the following resources to learn more about the process and how one can adapt it to different applications:
* [Overview of indexers](https://learn.microsoft.com/en-us/azure/search/search-indexer-overview)
* [Skill context and input annotation language](https://learn.microsoft.com/en-us/azure/search/cognitive-search-skill-annotation-language)
* [Reference inputs and outputs in skillsets](https://learn.microsoft.com/en-us/azure/search/cognitive-search-concept-annotations-syntax)

In [ ]:
# Create a skillset  
skillset_name = f"{index_name}-skillset"

split_skill = SplitSkill(  
    description="Split skill to chunk documents",  
    text_split_mode="pages",  
    context="/document",  
    maximum_page_length=300,  
    page_overlap_length=20,  
    inputs=[  
        InputFieldMappingEntry(name="text", source="/document/TextConcat"),  
    ],  
    outputs=[  
        OutputFieldMappingEntry(name="textItems", target_name="pages")  
    ]  
)

embedding_skill = AzureOpenAIEmbeddingSkill(  
    description="Skill to generate embeddings via Azure OpenAI",  
    context="/document/pages/*",  
    resource_uri=openai.api_base,  
    deployment_id=openai_deployment,  
    api_key=openai.api_key,  
    inputs=[  
        InputFieldMappingEntry(name="text", source="/document/pages/*"),  
    ],  
    outputs=[  
        OutputFieldMappingEntry(name="embedding", target_name="vector")  
    ]  
)  

index_projections = SearchIndexerIndexProjections(  
    selectors=[  
        SearchIndexerIndexProjectionSelector(  
            target_index_name=index_name,  
            parent_key_field_name="parent_id", # Note: this populates the "parent_id" search field
            source_context="/document/pages/*",  
            mappings=[  
                InputFieldMappingEntry(name="chunk", source="/document/pages/*"),
                InputFieldMappingEntry(name="vector", source="/document/pages/*/vector"),
                InputFieldMappingEntry(name="parent_product_id", source="/document/ProductId"),
                InputFieldMappingEntry(name="parent_text", source="/document/Text"),
                InputFieldMappingEntry(name="parent_summary", source="/document/Summary"),
                InputFieldMappingEntry(name="parent_score", source="/document/Score")
            ],  
        ),  
    ],
)  

skillset = SearchIndexerSkillset(  
    name=skillset_name,  
    description="Skillset to chunk documents and generating embeddings",  
    skills=[split_skill, embedding_skill],
    index_projections=index_projections  
)
  
client = SearchIndexerClient(service_endpoint, cogsearch_credential)  
client.create_or_update_skillset(skillset)  
print(f' {skillset.name} created')

 sqlserver-index-test-skillset created


## Create indexer

In [ ]:
# Create an indexer  
indexer_name = f"{index_name}-indexer"  

indexer = SearchIndexer(  
    name=indexer_name,  
    description="Indexer to chunk documents and generate embeddings",  
    skillset_name=skillset_name,  
    target_index_name=index_name,  
    data_source_name=data_source.name
)  
  
indexer_client = SearchIndexerClient(service_endpoint, cogsearch_credential)
indexer_result = indexer_client.create_or_update_indexer(indexer)  

# Run the indexer  
indexer_client.run_indexer(indexer_name)
print(f' {indexer_name} created')

 sqlserver-index-test-indexer created


In [ ]:
# Get the status of the indexer  
indexer_status = indexer_client.get_indexer_status(indexer_name)
print(f"Indexer status: {indexer_status.status}")

Indexer status: running


In [ ]:
# Allow some time for the indexer to process the data
import time
time.sleep(30)

# Use vector search for sample application

## Perform queries

In [ ]:
user_query = "ballons"

In the following output, we find the top 3 chunks that are most relevant to the user's query.

Feel free to retry the following cell in case of an empty response or a 429 error. An empty response probably indicates that the chunking/embedding process has not finished yet. A 429 error means there have been too many requests to the OpenAI embedding service and should go away on retrying.

In [ ]:
search_client = SearchClient(service_endpoint, index_name, credential=cogsearch_credential)
vector_query = VectorizableTextQuery(text=user_query, k_nearest_neighbors=3, fields="vector", exhaustive=True)
# Use the query below to pass in the raw vector query instead of the query vectorization
# vector_query = RawVectorQuery(vector=generate_embeddings(user_query), k=3, fields="vector")
  
results = search_client.search(
    search_text=None,  
    vector_queries=[vector_query],
    select=["Id", "parent_id", "chunk", "parent_product_id", "parent_text", "parent_summary", "parent_score"],
    top=3
)

for result in results:
    print(f"Search score: {result['@search.score']}")
    print(f"Parent Id: {result['parent_id']} | Chunk id: {result['Id']}")
    print(f"Product Id: {result['parent_product_id']}")
    print(f"Text chunk: {result['chunk']}") 
    print(f"Review summary: {result['parent_summary']}")
    print(f"Review text: {result['parent_text']}")
    print(f"Review score: {result['parent_score']}")
    print("-----")


Search score: 0.872132
Parent Id: 186 | Chunk id: bc8f505cc50b_186_pages_0
Product Id: P001
Text chunk: Summary: Fun and festive | Review: These balloons were perfect for our party. They lasted all day and were very colorful.
Review summary: Fun and festive
Review text: These balloons were perfect for our party. They lasted all day and were very colorful.
Review score: 5
-----
Search score: 0.8716175
Parent Id: 206 | Chunk id: bc8f505cc50b_206_pages_0
Product Id: P001
Text chunk: Summary: Lovely and charming | Review: These balloons were perfect for our party. They lasted all day and were very colorful.
Review summary: Lovely and charming
Review text: These balloons were perfect for our party. They lasted all day and were very colorful.
Review score: 5
-----
Search score: 0.8716175
Parent Id: 146 | Chunk id: bc8f505cc50b_146_pages_0
Product Id: P001
Text chunk: Summary: Lovely and charming | Review: These balloons were perfect for our party. They lasted all day and were very colorful.


## Generate GPT Response

### Prompt creation

In [ ]:
# create a prompt template 
template = """
    The user's query is: {query}
    The most relevant product review is: {context}
    The user is searching for a product matching their query. 
    Tell the user that after searching through our product database, you provide the reviews associated with the product. 
    Your answer should summarize the review text,
    include the product ID, and mention the score given in the review.
    """

In [ ]:
# create the context from the search response (requires regenerating results)
results = search_client.search(
    search_text=None,  
    vector_queries= [vector_query],
    select=["Id", "chunk", "parent_product_id", "parent_text", "parent_score"],
    top=3
)

context = ""
for result in results:
    context += f"Product id: {result['parent_product_id']}. Review text: {result['parent_text']}. Review score: {result['parent_score']}"
    
print(context)

Product id: P001. Review text: These balloons were perfect for our party. They lasted all day and were very colorful.. Review score: 5Product id: P001. Review text: These balloons were perfect for our party. They lasted all day and were very colorful.. Review score: 5Product id: P001. Review text: These balloons were perfect for our party. They lasted all day and were very colorful.. Review score: 5


In [ ]:
prompt = template.format(context=context, query=user_query)
print(prompt)


    The user's query is: ballons
    The most relevant product review is: Product id: P001. Review text: These balloons were perfect for our party. They lasted all day and were very colorful.. Review score: 5Product id: P001. Review text: These balloons were perfect for our party. They lasted all day and were very colorful.. Review score: 5Product id: P001. Review text: These balloons were perfect for our party. They lasted all day and were very colorful.. Review score: 5
    The user is searching for a product matching their query. 
    Tell the user that after searching through our product database, you provide the reviews associated with the product. 
    Your answer should summarize the review text,
    include the product ID, and mention the score given in the review.
    


### Call to OpenAI

In [ ]:
client = openai.AzureOpenAI(
    azure_endpoint=config["openai_api_base"],
    api_key=config["openai_api_key"],
    api_version="2024-02-01",
)

completion = client.chat.completions.create(
    model="gpt-35-turbo-16k",
    messages=[
        {
            "role": "user",
            "content": user_query,
        },
    ],
    extra_body={
        "data_sources":[
            {
                "type": "azure_search",
                "parameters": {
                    "endpoint": config["cogsearch_endpoint"],
                    "index_name": config["cogsearch_index_name"],
                    "authentication": {
                        "type": "api_key",
                        "key": config["cogsearch_api_key"],
                    }
                }
            }
        ],
    }
)

print(completion.model_dump_json(indent=2))

{
  "id": "b72983ed-ace7-4cc0-8d81-8220c66ee774",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The requested information is not available in the retrieved data. Please try another query or topic.",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null,
        "end_turn": true,
        "context": {
          "citations": [
            {
              "content": "Summary: Great for all kinds of parties | Review: These balloons were perfect for our party. They lasted all day and were very colorful.\n151\nP001\nGreat for all kinds of parties\nThese balloons were perfect for our party. They lasted all day and were very colorful.",
              "title": null,
              "url": null,
              "filepath": null,
              "chunk_id": "0"
            },
            {
              "content": "Summary: Great for all kinds of parties | Review: These balloons were p

# Notes

After finishing the sample, remember to delete unneeded resources:
* Table created within existing SQL SERVER DB
* Within the Search Service resource:
  * Data source connection
  * Index
  * Skillset
  * Indexer

These can always be recreated by rerunning the notebook.